In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
data_train = pd.read_json(open("./input/train.json"),'r')
data_test = pd.read_json(open("./input/test.json"),'r')
submitexam = pd.read_csv('./input/sample_submission.csv')

from bs4 import BeautifulSoup  
from collections import Counter

images_sample
sample_submission.csv
test.json
train.json



In [3]:
def get_most_common(dataset,c_name,n):
    cnt = Counter()
    datacolumn = dataset[c_name]
    #First Create a counter and find the top n most frequent words
    for eachr in datacolumn:
        if type(eachr)!= list:
            for single_word in eachr.split():
                cnt[single_word.lower()] +=1
        else:
            for word in eachr:
                for single_word in word.split():
                    cnt[single_word.lower()] +=1
    
    Most_Common_W = cnt.most_common(n)
    return Most_Common_W



def get_feature_matrix(Most_Common_W,dataset,c_name):
    n = len(Most_Common_W)
    datacolumn = dataset[c_name]
    #Then Create a data frame collecting 0-1 dummy variables
    dummy_data = np.zeros(shape=(len(datacolumn),n))
    d = pd.DataFrame(dummy_data, columns=dict(Most_Common_W).keys())
    
    for ieachr in range(len(datacolumn)):
        if ieachr %10000==0:
            print ieachr,'/',len(datacolumn)
        eachr = datacolumn.iloc[ieachr]
        if type(eachr) == list:
            for feature_word in ' '.join(datacolumn.iloc[ieachr]).lower().split():
                if feature_word in dict(Most_Common_W).keys():
                    d[feature_word].iloc[ieachr] = 1
        else:
            for feature_word in eachr.lower().split():
                if feature_word in dict(Most_Common_W).keys():
                    d[feature_word].iloc[ieachr] = 1
    return d



def get_numerical_feature(dataset, col_name):
    Newfeature = dataset[col_name]
    return Newfeature

def prob_to_csv(proba,colname,listing_id):
    pb = pd.DataFrame(proba)
    pb.columns = colname
    pb.to_csv("file_path.csv")



def find_freq_in_different_levels(data,level):
    data_low,nlow = find_freq(data[level =='low'])
    data_mid,nmid = find_freq(data[level =='medium'])
    data_high,nhigh = find_freq(data[level =='high'])
    return data_low,data_mid,data_high
    
        
    
def find_freq(data):
    cnt = Counter()
    ndata = len(data)
    for eachr in data:
        for single_word in eachr.split():
            cnt[single_word.lower()] +=1
    cnt = dict(cnt)
    return cnt,ndata


    

In [4]:

subcolumn_name = ['bathrooms','bedrooms','latitude','longitude','price']
N_feature = get_numerical_feature(data_train,subcolumn_name)
M_features = get_most_common(data_train,'features',500)
M_descrip = get_most_common(data_train,'description',500)
Dummy_feature = get_feature_matrix(M_features,data_train,'features')
Dummy_feature2 = get_feature_matrix(M_descrip,data_train,'description')
Dummy_feature.to_pickle('Dummy1.pkl')
Dummy_feature2.to_pickle('Dummy2.pkl')

0 / 49352
10000 / 49352
20000 / 49352
30000 / 49352
40000 / 49352
0 / 49352
10000 / 49352
20000 / 49352
30000 / 49352
40000 / 49352


In [83]:
Dummy_feature_test = get_feature_matrix(M_features,data_test,'features')
Dummy_feature2_test = get_feature_matrix(M_descrip,data_test,'description')

0 / 74659
10000 / 74659
20000 / 74659
30000 / 74659
40000 / 74659
50000 / 74659
60000 / 74659
70000 / 74659
0 / 74659
10000 / 74659
20000 / 74659
30000 / 74659
40000 / 74659
50000 / 74659
60000 / 74659
70000 / 74659


In [ ]:
Dummy_feature_test.to_pickle('Dummy_feature_test.pkl')
Dummy_feature2_test.to_pickle('Dummy_feature2_test.pkl')

In [76]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

class naive_bayes_trainer:
    
    class Data2:
        def __init__(self,feature,label):
            self.data = feature
            self.label = label
        def split_train_test(self):
            X_train, X_test, y_train, y_test = train_test_split(self.data.index, self.label,test_size=0.3, random_state=20)
            self.xtrain = self.data.iloc[X_train]
            self.xtest = self.data.iloc[X_test]
            self.y_train = y_train
            self.y_test = y_test
            
    def __init__(self,feature,label):
        self.data = self.Data2(feature,label)
        self.data.split_train_test()
    
    def naive_bayes_prob(self):
        BN_NB = MultinomialNB()
        self.model = BN_NB.fit(self.data.xtrain,self.data.y_train)
        return self.model
    
    def accuracy(self):
        y_pred = self.model.predict(self.data.xtest)
        matrix = confusion_matrix(self.data.y_test, y_pred)
        return matrix
    def predict(self, testdata):
        y_pred = self.model.predict(testdata)
        return y_pred
    
    
    

    

def t_to_csv(prob,cname,indexi):
    prob_matrix = pd.DataFrame(prob,index=indexi,columns=cname)
    prob_matrix.to_csv('./submit.csv',sep=",",index = True,index_label='listing_id')


In [92]:
Dummy = pd.concat([Dummy_feature2, Dummy_feature], axis=1)
NB = naive_bayes_trainer(Dummy,data_train['interest_level'])
NB.naive_bayes_prob()
NB.model

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [105]:
Dummy_test = pd.concat([Dummy_feature2_test, Dummy_feature_test], axis=1)
prob = NB.model.predict_proba(Dummy_test)
prob

array([[  2.32798005e-01,   3.24917605e-01,   4.42284390e-01],
       [  9.61009357e-02,   7.52421809e-01,   1.51477255e-01],
       [  3.28368269e-01,   3.40626344e-01,   3.31005387e-01],
       ..., 
       [  5.21538957e-03,   8.21059407e-01,   1.73725203e-01],
       [  9.31909715e-01,   8.24821151e-03,   5.98420737e-02],
       [  7.17202517e-06,   9.99992201e-01,   6.27131569e-07]])

In [109]:
t_to_csv(prob,['high','low','medium'],data_test.listing_id)